# Fault Classification Model

## Import Libraries

In [ ]:
import time
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pymongo import MongoClient
import json
import statistics
import numpy as np

import torch
import torch.nn as nn
#from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F

import mlflow.pytorch as mlp
from mlflow.tracking import MlflowClient
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import f1_score

# Data Preparation

In [ ]:
#Read data from pickle file containing the AE signals
original_ae = pd.read_pickle("./Pickle_files/ae_all_signals_july.pkl")

#remove columns where enough segment data is not present
mask = (original_ae['AE Signal RMS Abrichten Sp. 51'].str.len() > 500) & (original_ae['AE Signal RMS Abrichten Sp. 51'].str.len() < 750)
original_ae = original_ae.loc[mask]

original_ae

In [ ]:
#Since the diameter is stored as array of string, needs to be converted to array of floats
original_ae['Durchmesser Min. Fübo 1'] = original_ae['Durchmesser Min. Fübo 1'].str[0].apply(float)

In [ ]:
#create a column in dataframe stating whether a part is scrap or non-scrap
def label_race (row):
    if ((row['Durchmesser Min. Fübo 1'] >= 0.0008) | (row['Durchmesser Min. Fübo 1'] <= -0.0008)):
          return 1.0
    else:
        return 0.0
    
original_ae['is_scrap'] = original_ae.apply (lambda row: label_race(row), axis=1)
original_ae

In [ ]:
#Convert array of strings of AE signals to array of floats
original_ae["AE Signal RMS Abrichten Sp. 51"] = original_ae["AE Signal RMS Abrichten Sp. 51"].apply(lambda x : np.array([float(i) for i in x]))

In [ ]:
#resample rows to get equal scrap and non-scrap samples 
scrap_rows = original_ae.loc[original_ae['is_scrap'] == 1.0]
good_rows = original_ae.loc[original_ae['is_scrap'] == 0.0]   

sample_good_rows = good_rows.sample(frac=0.4, replace=True, random_state=1)
ae_aligned = pd.concat([scrap_rows, sample_good_rows])
ae_aligned

# pre-processing the AE signal

In [ ]:
def normalize(tSignal):
    # copy the data if needed, omit and rename function argument if desired
    signal = np.copy(tSignal) # signal is in range [a;b]
    signal -= np.min(signal) # signal is in range to [0;b-a]
    signal /= np.max(signal) # signal is normalized to [0;1]
    signal -= 0.5 # signal is in range [-0.5;0.5]
    signal *=2 # signal is in range [-1;1]
    return signal

In [ ]:
counter = 0
scrap_labels = []
arr = []

for ae_signal, scrap_label in zip(ae_aligned["AE Signal RMS Abrichten Sp. 51"], ae_aligned.is_scrap): 
    if len(ae_signal) < 625:
        counter = counter +1
        continue 
    else:
        #arr.append(normalize(ae_signal[0:625]))
        arr.append(ae_signal[0:625])
        scrap_labels.append(scrap_label)
            
arr2 = np.array(arr, dtype = np.float64)

In [ ]:
print(arr2.shape, len(scrap_labels))

# CNN model with Pytorch

In [ ]:
# Create 2 channels for input of CNN model 
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size1, size2):
    X=np.zeros((data.shape[0],size1, size2))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size1, size2))
    return X.astype(np.float16)


channel_mean=(mean(arr2,10)).astype(np.float16)
x_m=sig_image(channel_mean,25,25)
channel_median=(median(arr2,10)).astype(np.float16)
x_md=sig_image(arr2,25,25)
x_n = sig_image(arr2,25,25)


X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [ ]:
from sklearn.model_selection import train_test_split

scrap_labels = np.array(scrap_labels)
trainx, testx, trainlabel, testlabel = train_test_split(X, scrap_labels, test_size=0.2, random_state=20)

In [ ]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [ ]:
import torch.utils.data as data_utils
from torchsampler import ImbalancedDatasetSampler 

batch_size = 10 
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, sampler = ImbalancedDatasetSampler(train_tensor, trainlabel))

In [ ]:
batch_size = 10
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [ ]:
print(sig_train.size(), sig_test.size())

In [ ]:
#Normalise the signals if not done before
class Z_Normalisation(nn.Module):
    def __call__(self, tensor):
        #val = (tensor - tensor[0].mean()/ tensor[0].std()) #std normalisation -- option1
        return (tensor-torch.min(tensor))/(torch.max(tensor)-torch.min(tensor)) #min-max normalization -- option2
        #return val

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.sequential_stack = nn.Sequential(
            #Z_Normalisation(),
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4),
            nn.ReLU(),
            #nn.MaxPool2d(kernel_size=4),
        )

        self.fully_connected_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=256, out_features=256, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=256, out_features=64, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=64, out_features=2, bias=True),
            #nn.LogSoftmax()
        )
        self.gradients = None
        #self.classifier = nn.Linear(2 + 7) # 2 out-features and 7 hand made features
        
        
    def get_activations_gradient(self):
        return self.gradients

        # method for the activation exctraction
    def get_activations(self, x):
        return self.sequential_stack(x)

    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        y = self.sequential_stack(x)
        if y.requires_grad:
            h = y.register_hook(self.activations_hook)

        logits=self.fully_connected_stack(y)       
        return logits

In [ ]:
cnn = CNN().double()

In [ ]:
#weighted loss function when data is not sampled
#weights = torch.tensor([1.9856, 98.0144], dtype=torch.double)

# cross-entropy loss & Adam optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

### Training Step

In [ ]:
num_epochs = 500 #set epochs

total_step = len(train_loader)
loss_list = []
acc_list = []
with mlflow.start_run() as run:
    for epoch in range(num_epochs):
        for i, (signals, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            # Run the forward pass
            outputs = cnn(signals.double())
            loss = criterion(outputs, labels.long())

            loss_list.append(loss.item())
            # Backprop and perform Adam optimisation
            loss.backward()
            optimizer.step()
            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels.long()).sum().item()
            acc_list.append(correct / total)

            if (epoch+1) % 5 == 0 or epoch==0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
                
            mlph.log("loss", loss, on_epoch=True)
            mlp.log("train_acc", (correct / total), on_epoch=True)
            mlp.log_model(model, "model")

In [ ]:
#get average accuracy
avg_acc = np.mean(acc_list)
avg_acc

### Testing step

In [ ]:
#Use the test set to check the model accuracy
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

## Confusion matrix of binary classification (CNN Model)

In [ ]:
classes = {"Scrap", "Non-Scrap"}


y_pred = []
y_true = []

# iterate over test data
for inputs, labels in test_loader:
        output = cnn(inputs.double())

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth


# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
#df_cm = pd.DataFrame(cf_matrix , index = [i for i in classes],
#                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(cf_matrix, annot=True)

In [ ]:
#Calcualte F1-score
f1_score(y_true, y_pred, average='weighted')

## Feature extraction of AE signals

In [ ]:
#Thresholding Algorithm Source: 
#https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/

def thresholding_algo(y, lag = 2, threshold = 3, influence = 0.0):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]+0.3: 
                signals[i] = 1 #peak
            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return np.asarray(signals)

In [ ]:
weird_index = []
def calculate_features(signal, idx):

    result = thresholding_algo(signal) 
    
    # Get start, stop index pairs for islands/seq. of 1s
    idx_pairs = np.where(np.diff(np.hstack(([False],result==1,[False]))))[0].reshape(-1,2)
    
    if len(idx_pairs) == 0:
        weird_index.append(idx)
        return [None]
    
    # Get the island lengths, whose argmax would give us the ID of longest island.
    # Start index of that island would be the desired output
    start_longest_seq = idx_pairs[np.diff(idx_pairs,axis=1).argmax(),0]
    
    #index of the largest value
    highest_index = np.argmax(signal)
    
    #calculate angle of slope
    dx = (highest_index/len(signal) - start_longest_seq/len(signal))

    # Difference in y coordinates
    dy = 1 - signal[start_longest_seq]/signal[highest_index]

    # Angle between p1 and p2 in radians
    theta = math.atan2(dy, dx)
    
    #highest value
    max_value = signal[highest_index]

    #area under the curve
    area_under_curve = np.trapz(signal[signal < max_value])
    
    #last contact point
    occurences = np.where(result == 1)
    last_contact = occurences[0][-1]
    
    max_slope = max([x - z for x, z in zip(signal[:-1], signal[1:])])
    #print(signal[start_longest_seq], theta, max_value, area_under_curve, signal[last_contact], max_slope)
    return [signal[start_longest_seq], theta, max_value, area_under_curve, signal[last_contact], max_slope] 


In [ ]:
import math 
def feature_extract(data, size):
    features=np.zeros((data.shape[0], size))
    for i in range(data.shape[0]):
        features[i] = calculate_features(data[i], i)
    return features.astype(np.float16)

features_arr = feature_extract(data = arr2, size = 6)

## Data preparation for State-of-the-art algorithms

In [ ]:
pd.set_option('display.max_rows', 1200)
features = pd.DataFrame(list(map(np.ravel, features_arr)))
features.columns = ['first contact point', 'Theta', 'max value', 'area under curve', 'last contact point', 'slope']
features['is_scrap'] = ae_aligned["is_scrap"].to_list()
#features['Part_ID'] = ae_aligned.index.to_list()
X = features.dropna()
X

## Import for State-of-the-art algorithms

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import warnings

## Data Pre-processing

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X.loc[:, X.columns != 'is_scrap'])
X_norm = scaler.transform(X.loc[:, X.columns != 'is_scrap'])

## ML Models from scikit-learn

In [ ]:
accuracy_values = []
F1_score_values = []

In [ ]:
RF=RandomForestClassifier(n_estimators=10,random_state=10)
score = cross_val_score(RF, X_norm, X['is_scrap'], cv=10) #, scoring = 'f1' -- use for generating F1 score
mean_accuracy = np.mean(np.array(score))
print('Fold-wise accuracies: ', score)
print('Mean accuracy: ', mean_accuracy)
accuracy_values.append(mean_accuracy)
#F1_score_values.append(mean_accuracy)# when scoring = 'f1' is activated

In [ ]:
LR=LogisticRegression(random_state=6, solver='lbfgs',multi_class='multinomial')
score = cross_val_score(LR, X_norm, X['is_scrap'], cv=10) #, scoring = 'f1'
mean_accuracy = np.mean(np.array(score))
print('Fold-wise accuracies: ', score)
print('Mean accuracy: ', mean_accuracy)
accuracy_values.append(mean_accuracy)
#F1_score_values.append(mean_accuracy)# when scoring = 'f1' is activated

In [ ]:
svc=SVC(random_state=100, tol=1e-1) 
score = cross_val_score(svc, X_norm, X['is_scrap'], cv=5 )#, scoring = 'f1'
mean_accuracy = np.mean(np.array(score))
print('Fold-wise accuracies: ', score)
print('Mean accuracy: ', mean_accuracy)
accuracy_values.append(mean_accuracy)
#F1_score_values.append(mean_accuracy)# when scoring = 'f1' is activated

In [ ]:
LR=LogisticRegression(random_state=6, solver='lbfgs',multi_class='multinomial')
score = cross_val_score(LR, X_norm, X['is_scrap'], cv=10) #, scoring = 'f1'
mean_accuracy = np.mean(np.array(score))
print('Fold-wise accuracies: ', score)
print('Mean accuracy: ', mean_accuracy)
accuracy_values.append(mean_accuracy)
#F1_score_values.append(mean_accuracy)# when scoring = 'f1' is activated

In [ ]:
knn=KNeighborsClassifier(n_neighbors=25)
score = cross_val_score(knn, X_norm, X['is_scrap'], cv=10)#, scoring = 'f1'
mean_accuracy = np.mean(np.array(score))
print('Fold-wise accuracies: ', score)
print('Mean accuracy: ', mean_accuracy)
accuracy_values.append(mean_accuracy)
#F1_score_values.append(mean_accuracy)# when scoring = 'f1' is activated

## Plot accuracy and F1-score

In [ ]:
#fig = plt.figure()
#plt.figure(figsize = (15,7))
width = 0.35
ax = fig.add_axes([0,0,1,1])
algos = ['Random Forest Classifier', 'SVM', 'Logistic Regression', 'KNN Classifier', 'Multilayer Perceptron']

ax.set_ylabel('Mean Performance (%)')
ax.set_title('Performance of Classification Algorithms')

ax = plt.subplot(111)
plt.xticks(rotation=90)
acc = ax.bar(algos,accuracy_values, width, color='b')
f1 = ax.bar(algos,F1_score_values, width, color='g')

ax.legend( (acc, f1), ('accuracy', 'F1 score') )

'''for i, v in enumerate(accuracy_values):
    ax.text(i-.15, 
              v/accuracy_values[i]+82, 
              accuracy_values[i], 
              fontsize=5, 
              color='black')'''

plt.show()